# Train YOLOv8 Detection Models
**Author:** G8  
**Task:** 3.1 - Train YOLOv8 for Multi-Object Detection  
**Timeline:** Feb 8-9, 2025  

**Models to train:**
1. YOLOv8n (nano - fastest)
2. YOLOv8s (small - balanced)
3. YOLOv8m (medium - best accuracy)

## Setup and Imports

In [1]:
import os
import json
from pathlib import Path
import pandas as pd
import shutil
from ultralytics import YOLO

print("Libraries imported!")
print("Ultralytics YOLO ready")

/Users/kevin/Documents/GitHub/Python/VESKL/11.DAE/NEU/NEU_IE7615/Prj/Discriminative/G8/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Libraries imported!
Ultralytics YOLO ready


## Configuration

In [5]:
# Paths
PROJECT_ROOT = Path.cwd().parent if 'notebooks' in str(Path.cwd()) else Path.cwd()
DATA_YAML = PROJECT_ROOT / "data" / "data.yaml"
MODELS_PATH = PROJECT_ROOT / "models" / "detection"
LOGS_PATH = PROJECT_ROOT / "logs" / "detection"

MODELS_PATH.mkdir(parents=True, exist_ok=True)
LOGS_PATH.mkdir(parents=True, exist_ok=True)

# Training parameters
EPOCHS = 3  # Start with 10 for testing, increase to 150 for full training
IMG_SIZE = 640
BATCH_SIZE = 16
DEVICE = 'mps'  # Use MPS for Apple Silicon M1/M2/M3

print(f"Configuration:")
print(f"  Data config: {DATA_YAML}")
print(f"  Epochs: {EPOCHS}")
print(f"  Image size: {IMG_SIZE}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Device: {DEVICE} (Apple Silicon GPU)")

Configuration:
  Data config: /Users/kevin/Documents/GitHub/Python/VESKL/11.DAE/NEU/NEU_IE7615/Prj/Discriminative/G8/CNN_Attendance_System/data/data.yaml
  Epochs: 3
  Image size: 640
  Batch size: 16
  Device: mps (Apple Silicon GPU)


## Verify Dataset

In [3]:
print("="*80)
print("VERIFYING DATASET")
print("="*80)

# Check data.yaml exists
if not DATA_YAML.exists():
    print(f"ERROR: {DATA_YAML} not found!")
    print("Run Task 1.2 (multi-object generation) first")
else:
    print(f"✓ data.yaml found")
    
    # Read and display
    with open(DATA_YAML, 'r') as f:
        yaml_content = f.read()
    
    print("\nDataset configuration:")
    print(yaml_content[:500])  # Show first 500 chars
    
    # Count images
    multi_obj_path = PROJECT_ROOT / "data" / "multi_objects"
    train_imgs = len(list((multi_obj_path / "images" / "train").glob("*.jpg")))
    val_imgs = len(list((multi_obj_path / "images" / "val").glob("*.jpg")))
    test_imgs = len(list((multi_obj_path / "images" / "test").glob("*.jpg")))
    
    print(f"\nDataset size:")
    print(f"  Train: {train_imgs} images")
    print(f"  Val: {val_imgs} images")
    print(f"  Test: {test_imgs} images")
    print(f"  Total: {train_imgs + val_imgs + test_imgs} images")
    
    if train_imgs < 100:
        print("\n⚠️ WARNING: Too few training images. Run Task 1.2 first.")

VERIFYING DATASET
✓ data.yaml found

Dataset configuration:
# YOLOv8 Dataset Configuration
# Generated for CNN Attendance System Project

path: /Users/kevin/Documents/GitHub/Python/VESKL/11.DAE/NEU/NEU_IE7615/Prj/Discriminative/G8/CNN_Attendance_System/data/multi_objects
train: images/train
val: images/val
test: images/test

# Number of classes
nc: 35

# Class names
names:
  0: OBJ001
  1: OBJ002
  2: OBJ003
  3: OBJ004
  4: OBJ005
  5: OBJ006
  6: OBJ007
  7: OBJ008
  8: OBJ009
  9: OBJ010
  10: OBJ011
  11: OBJ016
  12: OBJ018
  13: OBJ019
  14: OBJ021

Dataset size:
  Train: 840 images
  Val: 180 images
  Test: 180 images
  Total: 1200 images


## Train Model 1: YOLOv8n (Nano - Fastest)

In [4]:
print("="*80)
print("TRAINING YOLOV8N (NANO)")
print("="*80)

# Load pretrained YOLOv8n
print("\nLoading YOLOv8n pretrained model...")
model_n = YOLO('yolov8n.pt')

# Train
print("\nStarting training...")
results_n = model_n.train(
    data=str(DATA_YAML),
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    device=DEVICE,
    patience=50,
    save=True,
    save_period=5,
    project=str(LOGS_PATH),
    name='train_n',
    exist_ok=True,
    pretrained=True,
    optimizer='Adam',
    verbose=True,
    seed=42,
    plots=True
)

# Copy best weights
best_n = LOGS_PATH / 'train_n' / 'weights' / 'best.pt'
shutil.copy2(best_n, MODELS_PATH / 'yolov8n_best.pt')

print("\n" + "="*80)
print("YOLOV8N TRAINING COMPLETED")
print("="*80)
print(f"Best model saved: {MODELS_PATH / 'yolov8n_best.pt'}")
print(f"Training results: {LOGS_PATH / 'train_n'}")

TRAINING YOLOV8N (NANO)

Loading YOLOv8n pretrained model...


100%|██████████| 6.23M/6.23M [00:00<00:00, 45.5MB/s]



Starting training...
New https://pypi.org/project/ultralytics/8.4.9 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.0 🚀 Python-3.9.6 torch-2.1.0 MPS (Apple M1)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/Users/kevin/Documents/GitHub/Python/VESKL/11.DAE/NEU/NEU_IE7615/Prj/Discriminative/G8/CNN_Attendance_System/data/data.yaml, epochs=10, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=5, cache=False, device=mps, workers=8, project=/Users/kevin/Documents/GitHub/Python/VESKL/11.DAE/NEU/NEU_IE7615/Prj/Discriminative/G8/CNN_Attendance_System/logs/detection, name=train_n, exist_ok=True, pretrained=True, optimizer=Adam, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7,

Unable to revert mtime: /Library/Fonts


Overriding model.yaml nc=80 with nc=35

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

...

WARNING ⚠️ NMS time limit 2.100s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  83%|████████▎ | 5/6 [00:34<00:06,  6.68s/it]

WARNING ⚠️ NMS time limit 1.500s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:42<00:00,  7.07s/it]


                   all        180       1140      0.309     0.0448     0.0644     0.0617
                OBJ001        180         26      0.238     0.0385     0.0296     0.0296
                OBJ002        180         32          1     0.0622      0.229      0.229
                OBJ003        180         29      0.397      0.069     0.0797     0.0797
                OBJ004        180         32       0.41      0.156      0.147      0.138
                OBJ005        180         37      0.466      0.135      0.129      0.117
                OBJ006        180         20          1          0      0.131      0.113
                OBJ007        180         35          1     0.0734      0.264      0.252
                OBJ008        180         29      0.305     0.0616     0.0591     0.0567
                OBJ009        180         38      0.577     0.0789      0.104     0.0964
                OBJ010        180         39      0.404      0.154      0.171      0.164
                OBJ01

## Train Model 2: YOLOv8s (Small - Balanced)

In [ ]:
print("="*80)
print("TRAINING YOLOV8S (SMALL)")
print("="*80)

print("\nLoading YOLOv8s pretrained model...")
model_s = YOLO('yolov8s.pt')

print("\nStarting training...")
results_s = model_s.train(
    data=str(DATA_YAML),
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    device=DEVICE,
    patience=50,
    save=True,
    save_period=5,
    project=str(LOGS_PATH),
    name='train_s',
    exist_ok=True,
    pretrained=True,
    optimizer='Adam',
    verbose=True,
    seed=42,
    plots=True
)

# Copy best weights
best_s = LOGS_PATH / 'train_s' / 'weights' / 'best.pt'
shutil.copy2(best_s, MODELS_PATH / 'yolov8s_best.pt')

print("\n" + "="*80)
print("YOLOV8S TRAINING COMPLETED")
print("="*80)
print(f"Best model saved: {MODELS_PATH / 'yolov8s_best.pt'}")

## Train Model 3: YOLOv8m (Medium - Best Accuracy)

In [ ]:
print("="*80)
print("TRAINING YOLOV8M (MEDIUM)")
print("="*80)

print("\nLoading YOLOv8m pretrained model...")
model_m = YOLO('yolov8m.pt')

print("\nStarting training...")
results_m = model_m.train(
    data=str(DATA_YAML),
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    device=DEVICE,
    patience=50,
    save=True,
    save_period=5,
    project=str(LOGS_PATH),
    name='train_m',
    exist_ok=True,
    pretrained=True,
    optimizer='Adam',
    verbose=True,
    seed=42,
    plots=True
)

# Copy best weights
best_m = LOGS_PATH / 'train_m' / 'weights' / 'best.pt'
shutil.copy2(best_m, MODELS_PATH / 'yolov8m_best.pt')

print("\n" + "="*80)
print("YOLOV8M TRAINING COMPLETED")
print("="*80)
print(f"Best model saved: {MODELS_PATH / 'yolov8m_best.pt'}")

## Training Summary

In [ ]:
print("="*80)
print("YOLOV8 TRAINING SUMMARY")
print("="*80)

# Collect results from training runs
summary_data = []

for variant in ['n', 's', 'm']:
    results_csv = LOGS_PATH / f'train_{variant}' / 'results.csv'
    if results_csv.exists():
        df_results = pd.read_csv(results_csv)
        last_epoch = df_results.iloc[-1]
        
        summary_data.append({
            'model': f'yolov8{variant}',
            'epochs_trained': len(df_results),
            'final_mAP50': float(last_epoch['metrics/mAP50(B)']),
            'final_mAP50_95': float(last_epoch['metrics/mAP50-95(B)']),
            'final_precision': float(last_epoch['metrics/precision(B)']),
            'final_recall': float(last_epoch['metrics/recall(B)'])
        })

df_summary = pd.DataFrame(summary_data)
df_summary = df_summary.sort_values('final_mAP50', ascending=False)

print("\n" + df_summary.to_string(index=False))

# Save summary
summary_path = LOGS_PATH / 'training_summary.csv'
df_summary.to_csv(summary_path, index=False)
print(f"\nSummary saved: {summary_path}")

# Best model
best = df_summary.iloc[0]
print("\n" + "="*80)
print("BEST MODEL")
print("="*80)
print(f"Model: {best['model']}")
print(f"mAP50: {best['final_mAP50']:.4f}")
print(f"mAP50-95: {best['final_mAP50_95']:.4f}")

if best['final_mAP50'] >= 0.85:
    print("\n✅ SUCCESS: Target mAP50 (>0.85) achieved!")
else:
    print(f"\n⚠️ Current mAP50: {best['final_mAP50']:.3f}, target: 0.85")
    print("Consider increasing EPOCHS to 150 for better results")

## View Training Results

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

print("="*80)
print("TRAINING RESULTS VISUALIZATION")
print("="*80)

# Display training curves for best model
best_variant = best['model'][-1]  # Get 'n', 's', or 'm'
results_img = LOGS_PATH / f'train_{best_variant}' / 'results.png'

if results_img.exists():
    print(f"\nDisplaying results for {best['model']}:")
    img = mpimg.imread(results_img)
    fig, ax = plt.subplots(figsize=(16, 10))
    ax.imshow(img)
    ax.axis('off')
    ax.set_title(f"Training Results - {best['model']}", fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    print("Results image not found")

## Task 3.1 Complete!

**Deliverables:**
- 3 trained YOLOv8 models (.pt files)
- Training logs and plots
- Results CSV files

**Next:** Task 3.2 - YOLOv8 evaluation (Hassan)

**Note:** If mAP50 <0.85 after 10 epochs, change EPOCHS to 150 and retrain